In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from datetime import datetime
import threading

In [2]:
jumlah_index = 10
threads_link = []
links = []
results = []
threads = []
keywords = 'ganjar'

In [3]:
def scrape_links(page_number,keywords):
    url=f"https://www.beritajakarta.id/news_index/search?q={keywords}&p={page_number}"
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")
    articles = soup.find_all('div',{"class": "media blog-list news-index"})
    
    page_links = []
    for article in articles:
        link = article.find('a')['href']
        page_links.append(link)
    
    print(f"Scraped {len(page_links)} links from page {page_number}")
    return page_links

In [4]:
for page_number in range(1, jumlah_index + 1):
    thread = threading.Thread(target=lambda p=page_number: links.extend(scrape_links(p,keywords)))
    thread.start()
    threads_link.append(thread)

for thread in threads_link:
    thread.join()
print("Total Links:", len(links))

Scraped 10 links from page 5
Scraped 10 links from page 4
Scraped 10 links from page 1
Scraped 10 links from page 2
Scraped 10 links from page 3
Scraped 10 links from page 7
Scraped 10 links from page 9
Scraped 10 links from page 10
Scraped 10 links from page 6
Scraped 10 links from page 8
Total Links: 100


In [5]:
def scrape_url(url,keywords):
    try:
        response = requests.get(url)
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')        
            # Judul Berita
            title_elem = soup.find('h1')
            if title_elem:
                title_text = title_elem.text
            else:
                title_text = "Title not found"
                
            elem = soup.find('div', {"class": "text-secondary"})
            
            if elem:
                input_string = elem.text
                parts = input_string.split("Reporter :")
                date_text = parts[0].strip()
                editor_and_view = parts[1].split("Editor :")
                author_text = editor_and_view[0].strip()
                remaining_text = editor_and_view[1]
                editor_parts = remaining_text.split()
                editor_text = editor_parts[0]
                viewer_text = editor_parts[-1]
            else:
                author_text = "Author not found"
                date_text = "Date not found"
                editor_text = "Editor not found"
                viewer_text = "Viewer not found"
                
                
            # Author berita
            # author_elem = soup.find('div', {"class": "jeg_meta_author"})
            # if author_elem:
            #     author_text = author_elem.find('a')
            #     author_text = author_text.text
            # else:
            #     author_text = "Author not found"
                     
            # tanggal berita
            # date_elem = soup.find('div', {"class": "jeg_meta_date"})
            # if date_elem:
            #     date_text = date_elem.find('a')
            #     date_text = date_text.text
            # else:
            #     date_text = "Date not found"
            # Category berita
            category_elements = soup.find_all('li', {"class":"breadcrumb-item"}) 
            if category_elements:
                category_text= category_elements[1].text
            else:
                category_text = "Category not found"
            #     # Content Berita
            body_elem = soup.find('div', {"class": "news-content"})
            
            if body_elem:
                content_elem = body_elem.find_all('p')
                content_text = ""
                for p in content_elem:
                    content_text += p.text.strip() + "\n"
                
                if content_text.strip():
                    content_text=content_text
                else:
                    content_text ="Content not found"
            else:
                content_text ="Content not found"

            results.append({'title': title_text,
                            'keywords':keywords,
                            'author' : author_text,
                            'category':category_text,
                            'editor' : editor_text,
                            'view' : viewer_text,
                            'date': date_text,
                            'content' : content_text,
                            'region':'jakarta',
                            'link' : url})
        else:
            print(f"Failed to retrieve data from {url}")

    except requests.exceptions.RequestException as e:
        print(f"Error fetching URL '{url}': {e}")
    except Exception as e:
        print(f"Error processing URL '{url}': {e}")

In [6]:
for url in links:
    thread = threading.Thread(target=scrape_url, args=(url,keywords))
    thread.start()
    threads.append(thread)
    
for thread in threads:
    thread.join()

Failed to retrieve data from https://www.beritajakarta.id/read/32589/pns-dilarang-gunakan-kendaraan-dinas-untuk-mudik
Failed to retrieve data from https://www.beritajakarta.id/read/47879/langgar-trotoar-10-kendaraan-di-jl-dewi-sartika-ditindak
Failed to retrieve data from https://www.beritajakarta.id/read/80327/10-pelanggar-psbb-di-kemayoran-disanksi-kerja-sosial


In [7]:
df = pd.DataFrame(results)
print("Total Hasil ",len(results))
df.head(10)

Total Hasil  97


,title,keywords,author,category,editor,view,date,content,region,link
0,51 Pelanggar PSBB di Setiabudi Ditertibkan,ganjar,TP Moan Simanjuntak,Lintas Kota,Andry,970,"Jumat, 07 Agustus 2020 15:47",51 pelanggar aturan Pembatasan Sosial Berskala...,jakarta,https://www.beritajakarta.id/read/81900/51-pel...
1,Korban Tenggelam di Kali Cipinang Ditemukan Tewas,ganjar,Nurito,Lintas Kota,Rio,7126,"Sabtu, 20 Agustus 2016 22:37",Seorang pengendara sepeda motor tergelincir da...,jakarta,https://www.beritajakarta.id/read/35341/korban...
2,45 Pelanggar PSBB di Pasar Minggu Ditindak,ganjar,TP Moan Simanjuntak,Lintas Kota,Andry,1492,"Jumat, 07 Agustus 2020 17:04",Penindakan terhadap pelanggar aturan Pembatasa...,jakarta,https://www.beritajakarta.id/read/81907/45-pel...
3,17 Pelanggar PSBB di Gambir Disanksi Kerja Sosial,ganjar,Adriana Megawati,Lintas Kota,Andry,1091,"Rabu, 05 Agustus 2020 16:51",17 pelanggar aturan Pembatasan Sosial Berskala...,jakarta,https://www.beritajakarta.id/read/81827/17-pel...
4,Perusahaan Penyebab Polusi Debu Batu Bara di M...,ganjar,Aldi Geri Lumban Tobing,Pemerintahan,Andry,2347,"Selasa, 15 Maret 2022 16:27",Dinas Lingkungan Hidup (LH) DKI Jakarta mengam...,jakarta,https://www.beritajakarta.id/read/95958/perusa...
5,13 Warga di Kebayoran Lama Terjaring Operasi T...,ganjar,TP Moan Simanjuntak,Lintas Kota,Andry,1273,"Senin, 16 November 2020 18:38",13 warga di dua lokasi berbeda di wilayah Keba...,jakarta,https://www.beritajakarta.id/read/84783/13-war...
6,3.904 Pelanggar PSBB di Jakut Ditindak,ganjar,Budhi Firmansyah Surapati,Lintas Kota,Andry,1153,"Senin, 12 Oktober 2020 19:02","Selama empat pekan belakangan, tercatat ada 3....",jakarta,https://www.beritajakarta.id/read/83766/3904-p...
7,64 Kendaraan Parkir Liar di Jl Budi Mulia Diti...,ganjar,Budhi Firmansyah Surapati,Lintas Kota,Andry,2422,"Selasa, 11 Februari 2020 20:57",44 petugas gabungan menggelar penertiban parki...,jakarta,https://www.beritajakarta.id/read/76950/64-ken...
8,PAM Jaya KomitmenTingkatkan Keterbukaan Inform...,ganjar,Anita Karyati,BUMD,Budhy,2052,"Kamis, 29 Desember 2022 07:49",Tekad PAM Jaya untuk terus menghadirkan inovas...,jakarta,https://www.beritajakarta.id/read/114739/pam-j...
9,Tujuh Personel Dinas Gulkarmat Berprestasi Dig...,ganjar,Aldi Geri Lumban Tobing,Pemerintahan,Erikyanri,1886,"Senin, 09 Desember 2019 12:40",Tujuh personel Dinas Penanggulangan Kebakaran ...,jakarta,https://www.beritajakarta.id/read/75100/tujuh-...


In [8]:
filtered_df = df[df['content'].str.contains(keywords, case=False)|
                 df['title'].str.contains(keywords, case=False)]
print('hasil filter',len(filtered_df))
filtered_df.head(10)

hasil filter 97


,title,keywords,author,category,editor,view,date,content,region,link
0,51 Pelanggar PSBB di Setiabudi Ditertibkan,ganjar,TP Moan Simanjuntak,Lintas Kota,Andry,970,"Jumat, 07 Agustus 2020 15:47",51 pelanggar aturan Pembatasan Sosial Berskala...,jakarta,https://www.beritajakarta.id/read/81900/51-pel...
1,Korban Tenggelam di Kali Cipinang Ditemukan Tewas,ganjar,Nurito,Lintas Kota,Rio,7126,"Sabtu, 20 Agustus 2016 22:37",Seorang pengendara sepeda motor tergelincir da...,jakarta,https://www.beritajakarta.id/read/35341/korban...
2,45 Pelanggar PSBB di Pasar Minggu Ditindak,ganjar,TP Moan Simanjuntak,Lintas Kota,Andry,1492,"Jumat, 07 Agustus 2020 17:04",Penindakan terhadap pelanggar aturan Pembatasa...,jakarta,https://www.beritajakarta.id/read/81907/45-pel...
3,17 Pelanggar PSBB di Gambir Disanksi Kerja Sosial,ganjar,Adriana Megawati,Lintas Kota,Andry,1091,"Rabu, 05 Agustus 2020 16:51",17 pelanggar aturan Pembatasan Sosial Berskala...,jakarta,https://www.beritajakarta.id/read/81827/17-pel...
4,Perusahaan Penyebab Polusi Debu Batu Bara di M...,ganjar,Aldi Geri Lumban Tobing,Pemerintahan,Andry,2347,"Selasa, 15 Maret 2022 16:27",Dinas Lingkungan Hidup (LH) DKI Jakarta mengam...,jakarta,https://www.beritajakarta.id/read/95958/perusa...
5,13 Warga di Kebayoran Lama Terjaring Operasi T...,ganjar,TP Moan Simanjuntak,Lintas Kota,Andry,1273,"Senin, 16 November 2020 18:38",13 warga di dua lokasi berbeda di wilayah Keba...,jakarta,https://www.beritajakarta.id/read/84783/13-war...
6,3.904 Pelanggar PSBB di Jakut Ditindak,ganjar,Budhi Firmansyah Surapati,Lintas Kota,Andry,1153,"Senin, 12 Oktober 2020 19:02","Selama empat pekan belakangan, tercatat ada 3....",jakarta,https://www.beritajakarta.id/read/83766/3904-p...
7,64 Kendaraan Parkir Liar di Jl Budi Mulia Diti...,ganjar,Budhi Firmansyah Surapati,Lintas Kota,Andry,2422,"Selasa, 11 Februari 2020 20:57",44 petugas gabungan menggelar penertiban parki...,jakarta,https://www.beritajakarta.id/read/76950/64-ken...
8,PAM Jaya KomitmenTingkatkan Keterbukaan Inform...,ganjar,Anita Karyati,BUMD,Budhy,2052,"Kamis, 29 Desember 2022 07:49",Tekad PAM Jaya untuk terus menghadirkan inovas...,jakarta,https://www.beritajakarta.id/read/114739/pam-j...
9,Tujuh Personel Dinas Gulkarmat Berprestasi Dig...,ganjar,Aldi Geri Lumban Tobing,Pemerintahan,Erikyanri,1886,"Senin, 09 Desember 2019 12:40",Tujuh personel Dinas Penanggulangan Kebakaran ...,jakarta,https://www.beritajakarta.id/read/75100/tujuh-...


save data

In [9]:
current_datetime = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
excel_file_name = f'../../tempat_hasil_daerah/beritaJakarta_{current_datetime}.xlsx'
filtered_df.to_excel(excel_file_name, index=False)

print(f'Data has been saved to {excel_file_name}')

Data has been saved to ../../tempat_hasil_daerah/beritaJakarta_2023-10-12_17-53-43.xlsx
